# TP4 - Exercício 1
## Grupo 1

*   Diogo Coelho da Silva A100092
*   Pedro Miguel Ramôa Oliveira A97686


#### 1. Import's

In [17]:
from z3 import *

#### 2.Definição do algoritmo de euclides



In [18]:
def algoritmo_de_euclides(a, b):
    r1, r2, s1, s2, t1, t2 = a, b, 1, 0, 0, 1
    print(f"Argumentos de entrada: a: {a}, b: {b}, r1: {r1}, r2: {r2}, s1: {s1}, s2: {s2}, t1: {t1}, t2: {t2}")

    while r2 != 0:
        quociente = r1 // r2
        r1, r2 = r2, r1 - quociente * r2
        s1, s2 = s2, s1 - quociente * s2
        t1, t2 = t2, t1 - quociente * t2

        print(f"Iteração: a: {a}, b: {b}, r1: {r1}, r2: {r2}, s1: {s1}, s2: {s2}, t1: {t1}, t2: {t2}, quociente: {quociente}")

    print(f"Resultado final: a: {a}, b: {b}, r1: {r1}, r2: {r2}, s1: {s1}, s2: {s2}, t1: {t1}, t2: {t2}")
    print(f"Máximo divisor comum (MDC): {r1}\n")
    return r1

algoritmo_de_euclides(12, 18)

Argumentos de entrada: a: 12, b: 18, r1: 12, r2: 18, s1: 1, s2: 0, t1: 0, t2: 1
Iteração: a: 12, b: 18, r1: 18, r2: 12, s1: 0, s2: 1, t1: 1, t2: 0, quociente: 0
Iteração: a: 12, b: 18, r1: 12, r2: 6, s1: 1, s2: -1, t1: 0, t2: 1, quociente: 1
Iteração: a: 12, b: 18, r1: 6, r2: 0, s1: -1, s2: 3, t1: 1, t2: -2, quociente: 2
Resultado final: a: 12, b: 18, r1: 6, r2: 0, s1: -1, s2: 3, t1: 1, t2: -2
Máximo divisor comum (MDC): 6



6

In [19]:
from z3 import *


def prove(f):
    with Solver() as s:
        s.add(Not(f))
        if s.check() == sat:
            print("Failed to prove.")
        else:
            print("Proved.")

a = Int('a')
b = Int('b')
r = Int('r')
rr = Int('rr')
s = Int('s')
ss = Int('ss')
t = Int('t')
tt = Int('tt')
q = Int('q')

gcd = Function('gcd', IntSort(), IntSort(), IntSort())
gcdAx1 = ForAll([a], Implies(a >= 0, gcd(a, 0) == a))
gcdAx2 = ForAll([b], Implies(b >= 0, gcd(0, b) == b))
gcdAx3 = ForAll([a, b], Implies(And(a > 0, b > 0), gcd(a, b) == gcd(b, a % b)))

gcdF = And(gcdAx1, gcdAx2, gcdAx3)

preCond = And(a > 0, b > 0)

axioms = And(
    r == a,
    rr == b,
    s == 1,
    ss == 0,
    t == 0,
    tt == 1
)

pre = And(preCond, axioms)

inv = And(
    gcd(a, b) == gcd(r, rr),
    r == a * s + b * t,
    rr == a * ss + b * tt
)

preFinal = (Implies(pre, inv))

exitCond = ForAll([r,rr,s,ss,t,tt], And((rr == 0), inv))

posCond = And(
    r == gcd(a, b),
    r == a * s + b * t
)

posFinal = Implies(exitCond, posCond)

spc = And(preFinal, posFinal)

prove(Implies(gcdF, spc))

Z3Exception: b'Overflow encountered when expanding vector'